In [1]:
# libs para processamento
from datetime import date
import json

import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path

from hidromet.INMET import INMET
from hidromet.utils import dir_projeto
from shapely.geometry import Point
from hidromet import limpeza
from hidromet import modelos

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
diretorio = dir_projeto()

# Obter inventário de postos do INMET

In [3]:
inmet = INMET()
estacoes = inmet.listar_estacoes()
df_estacoes = pd.DataFrame(estacoes)

df_estacoes.loc[:, ["VL_LATITUDE", "VL_LONGITUDE"]] = df_estacoes.loc[:, ["VL_LATITUDE", "VL_LONGITUDE"]].apply(pd.to_numeric)

# Recortar postos dentro do contorno das bacias

## Tratamento dos contornos

In [4]:
contornos = diretorio.joinpath("contornos", "iguacu.shp")

gdf = gpd.read_file(contornos)
crs = gdf.crs

In [5]:
coords = [Point(x) for x in zip(df_estacoes.VL_LONGITUDE,
                                df_estacoes.VL_LATITUDE)]

postos_plu_inmet = gpd.GeoDataFrame(df_estacoes, geometry = coords, crs = crs)

## Postos do INMET dentro dos contornos

In [6]:
bacias = list()
for ix in gdf.index:
    geometria = gdf.loc[ix, 'geometry']
    bacia = gdf.loc[ix, 'bacia']
    recorte_postos_inmet = postos_plu_inmet.geometry.within(geometria)
    postos_inmet_dentro = postos_plu_inmet.loc[recorte_postos_inmet]
    postos_inmet_dentro = postos_inmet_dentro.assign(bacia=bacia)

    bacias.append(postos_inmet_dentro)



In [7]:
df_postos_bacias = pd.concat(bacias)

In [8]:
dir_postos = diretorio.joinpath("arquivos", "postos_plu_inmet_smap.csv")
df_postos_bacias.to_csv(dir_postos)

## Obter séries de chuva por bacia

In [9]:
data_inicial="1980-01-01"
data_final=date.today().strftime("%Y-%m-%d")

dir_prec = diretorio.joinpath("arquivos", "series-chuva-inmet")
dir_prec.mkdir(parents=True, exist_ok=True)

bacias_gp = df_postos_bacias.groupby("bacia")
bacias = bacias_gp.groups.keys()

info_bacias = list()
saida_previsao = list()
for bacia in bacias:
    print(f"Obtendo séries de chuva para postos na bacia {bacia}.")
    bacia_df = bacias_gp.get_group(bacia)
    n_postos = len(bacia_df.index)

    chuvas_bacia = list()
    postos_sem_dado = list()
    postos_com_falha = list()
    postos_nao_representativos = list()
    postos_ok = list()
    for estacao in bacia_df.index:
        posto = bacia_df.loc[estacao, "CD_ESTACAO"]
        estacao_json = inmet.obter_dados_estacao(cod_estacao=posto, data_inicial=data_inicial, data_final=data_final, freq="D")
        df = pd.DataFrame(estacao_json)
        df.rename({"DT_MEDICAO": "data", "CHUVA": posto}, axis=1, inplace=True)
        df.set_index("data", inplace=True)
        df.index = pd.to_datetime(df.index)

        df = df[[posto]].apply(pd.to_numeric)


        df = limpeza.remover_duplicados(df)
        df[posto] = limpeza.recortar_periodo(
            serie=df[posto], 
            inicio=pd.to_datetime("20000101"), 
            fim=pd.Timestamp('today')
            )

        df.dropna(inplace=True)
        
        df[posto] = limpeza.remover_outliers(df[posto], p=0.99)
        df[posto] = limpeza.substituir_negativos(df[posto])
        falhas = limpeza.contabilizar_falhas(df[posto])
        df = limpeza.remover_meses_nao_representativos(df)
        anos_disponiveis = limpeza.anos_disponiveis(df)
        n_dados = len(df)

        info_posto = modelos.Posto(
            latitude=bacia_df.loc[estacao, 'VL_LATITUDE'],
            longitude=bacia_df.loc[estacao, 'VL_LONGITUDE'],
            codigo=posto,
            n_dados=n_dados
        )


        if not df.empty and falhas < 0.15 and anos_disponiveis > 8:
            postos_ok.append(info_posto)
            chuvas_bacia.append(df)
        elif df.empty:
            postos_sem_dado.append(info_posto)
        elif anos_disponiveis < 8: 
            postos_nao_representativos.append(info_posto)
        elif falhas > 0.15:
            postos_com_falha.append(info_posto)


    info_bacia = modelos.Bacia(
        serie_vazia=postos_sem_dado,
        nao_representativos=postos_nao_representativos,
        com_falhas=postos_com_falha,
        postos_ok=postos_ok,
        n_postos=n_postos,
        bacia=bacia
    )


    info_bacias.append(info_bacia)
    try:
        df_chuvas_bacia = pd.concat(chuvas_bacia, axis=1)
        arquivo_bacia = dir_prec.joinpath(f"{bacia}.csv")
        df_chuvas_bacia.to_csv(arquivo_bacia)
    except (ValueError):
        print(
            f"Não há séries de precipitação disponíveis para os postos encontrados na bacia {bacia}"
        )

Obtendo séries de chuva para postos na bacia Iguaçú.


In [10]:
for info in info_bacias:
    arquivo_json = Path(dir_prec, f'coords_{info.bacia}.json')
    with open(arquivo_json, 'w') as f:
        json.dump(info.dict(), f)